# Hello Triangle

Welcome to your very first tutorial on rendering opengl using pyside2.

What we are going to do now is pretty simple. We shall draw the `Hello Triangle` more specifically a single yellow triangle.
You shall see that once you can draw a triangle, or better a cube, in a graphics api, you would understand the stages of rendering pretty well.

Now run the cell below to see the final version of our triangle

In [ ]:
import subprocess

subprocess.run(["python", "./app.py"])

Cool eh!

Now let's do a fresh start and create a directory under this one called `myTriangle`.

- Open the terminal
- Go to this directory `cd PATH_TO_GITPROJECT/tutorials/01-triangle/`
- Create a new directory `mkdir myTriangle`
- Inside create to files `touch app.py` and `touch myGLWidget.py`

The `app.py` will hold the window in which the gl widget would live, and `myGLWidget.py` simply defines the OpenGL rendering widget.

Let's start by filling the easy one the `app.py`

### Window Containing the OpenGL Rendering Widget

This is essentially no different from a window containing any other widget, which is one of the reasons why you might want to use `pyside2` in the first place. It gives you to capacity to mix up an opengl widget with other traditional ones.

Here is the list of things we would need:

In [ ]:
from PySide2 import QtWidgets, QtCore, QtGui
from myGLWidget import TriangleGL
import sys

Obviously we have not made a TriangleGL widget yet, but we do need the other ones.

Just to show that we can actually mix other widgets in this window. 

Here is a small function that creates sliders.

In [ ]:
def createSlider():
    "Create vertical slider widget"
    slider = QtWidgets.QSlider(QtCore.Qt.Vertical)

    slider.setRange(0, 360 * 16)
    slider.setSingleStep(16)
    slider.setPageStep(15 * 16)
    slider.setTickInterval(15 * 16)
    slider.setTickPosition(QtWidgets.QSlider.TicksRight)
    return slider

Here is the constructor of our application

In [ ]:
class AppWindow(QtWidgets.QMainWindow):
    "Application window"

    def __init__(self, parent=None):
        super().__init__(parent)
        self.glWidget = TriangleGL()  # initialized the gl widget
        self.xSlider = createSlider() # some slider
        self.ySlider = createSlider() # some slider
        self.zSlider = createSlider() # some slider

        mainLayout = QtWidgets.QHBoxLayout()  # a layout to hold everything together
        
        # adding widgets to layout 
        mainLayout.addWidget(self.glWidget) # 
        mainLayout.addWidget(self.xSlider)
        mainLayout.addWidget(self.ySlider)
        mainLayout.addWidget(self.zSlider)
        
        # main parent/central widget to hold the layout
        self.mainwidget = QtWidgets.QWidget()
        self.mainwidget.setLayout(mainLayout)
        
        # Its parent is the application window
        self.mainwidget.setParent(self)
        
        # Setting the title of the window
        self.setWindowTitle("Triangle Opengl widget")
        
        # setting its minimum size 
        # do not forget a resize event does not 
        # necessarily trigger a resize for the widgets inside
        self.setMinimumSize(800, 600)

We also add keypress method to easily close the application.

In [ ]:
    def keyPressEvent(self, event):
        if event.key() == QtCore.Qt.Key_Escape:
            self.close()
        else:
            super().keyPressEvent(event)

So our final window looks like the following

In [ ]:
from PySide2 import QtWidgets, QtCore, QtGui
from myGLWidget import TriangleGL
import sys

def createSlider():
    slider = QtWidgets.QSlider(QtCore.Qt.Vertical)

    slider.setRange(0, 360 * 16)
    slider.setSingleStep(16)
    slider.setPageStep(15 * 16)
    slider.setTickInterval(15 * 16)
    slider.setTickPosition(QtWidgets.QSlider.TicksRight)
    return slider


class AppWindow(QtWidgets.QMainWindow):
    "Application window"

    def __init__(self, parent=None):
        super().__init__(parent)
        self.glWidget = TriangleGL()
        self.xSlider = createSlider()
        self.ySlider = createSlider()
        self.zSlider = createSlider()

        mainLayout = QtWidgets.QHBoxLayout()
        mainLayout.addWidget(self.glWidget)
        mainLayout.addWidget(self.xSlider)
        mainLayout.addWidget(self.ySlider)
        mainLayout.addWidget(self.zSlider)
        self.mainwidget = QtWidgets.QWidget()
        self.mainwidget.setLayout(mainLayout)
        self.mainwidget.setParent(self)
        self.setWindowTitle("Triangle Opengl widget")
        self.setMinimumSize(800, 600)

    def keyPressEvent(self, event):
        if event.key() == QtCore.Qt.Key_Escape:
            self.close()
        else:
            super().keyPressEvent(event)


if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    window = AppWindow()
    window.show()
    res = app.exec_()
    sys.exit(res)

The last part with `if __name__ == '__main__'` is for dealing with `import` statements.

As you can see it is a fairly simple window which contains 3 sliders and an opengl widget.

Now let's create our OpenGL widget